In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
path = '/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/ToDatabase/'

In [3]:
df_address = pd.read_csv(path+'addr_junct_table.csv')

In [4]:
df_blockgroup = pd.read_csv(path+'CensusBlock_v02.csv')

In [5]:
df_tract = pd.read_csv(path+'CensusTract_v02.csv')

In [6]:
def create_blockgroup_spec(x):
    a = x.iloc[0]
    b = x.iloc[1]
    if not math.isnan(a) and not math.isnan(b):
        return str(int(a)) + "_" + str(int(b))

In [7]:
df_address.drop('Id', inplace=True, axis=1)
df_address.insert(0, 'AddressID', [i for i in range(1, len(df_address)+1)])
df_blockgroup.insert(0, 'BlockGroupID', [i for i in range(1, len(df_blockgroup)+1)])
df_bg_spec = df_blockgroup[['BlockGroup', 'Tract']]
df_addr_bg_spec = df_address[['BlockGroup', 'Tract']]
df_blockgroup.insert(1, 'BlockGroupCombo', df_bg_spec.apply(create_blockgroup_spec, axis=1))
df_address.insert(6, 'BlockGroupCombo', df_addr_bg_spec.apply(create_blockgroup_spec, axis=1))
df_address[['Tract', 'BlockGroup', 'Block']] = df_address[['Tract', 'BlockGroup', 'Block']].astype('Int64')
lis = [556, "", np.nan, np.nan, np.nan, np.nan, "NOT FOUND"]
lis.extend([np.nan]*98)
not_found_entry = pd.DataFrame([lis], columns=list(df_blockgroup))
df_blockgroup = df_blockgroup.append(not_found_entry, ignore_index=True)

In [8]:
junction_l = []
for i in range(len(df_address)):
    if i % 10000 == 0:
        print(i)
    addr_record = df_address.iloc[i]
    curr_combo = addr_record['BlockGroupCombo']
    allmatches = df_blockgroup['BlockGroupCombo'][df_blockgroup['BlockGroupCombo'] == curr_combo].index
    if len(allmatches) == 0:
        junction_l.append(([i+1], [556]))
    else:
        junction_l.append((([i+1]*len(allmatches)), allmatches))
junction_one = []
junction_two = []
for i in range(len(junction_l)):
    to_append = junction_l[i]
    junction_one.append(to_append[0])
    junction_two.append(to_append[1])
junction_one = [x for lst in junction_one for x in lst]
junction_two = [x for lst in junction_two for x in lst]
junction = pd.DataFrame({'AddressID':junction_one, 'BlockGroupID':junction_two})

0
10000
20000
30000


In [16]:
junction.head()

,AddressID,BlockGroupID
0,1,56
1,1,167
2,1,278
3,1,389
4,1,500


NameError: name 'df_bg' is not defined

In [13]:
df_blockgroup.head()

,BlockGroupID,BlockGroupCombo,GEO.id,Year,GEO.id2,Tract,BlockGroup,County,TotalMale,MaleUnder5,...,HispanicPacificIslander,HispanicOtherRace,Hispanic2Plus,Hispanic2PlusIncOther,Hispanic2PlusExcOther,TotalHomes,TotalOccupiedHomes,TotalVacantHomes,TotalOwnedHomes,TotalRentedHomes
0,1,1_100,1500000US130950001001,2013.0,1.309500e+11,100.0,1,Dougherty County,807.0,58.0,...,0.0,0.0,0.0,0.0,0.0,512.0,512.0,0.0,357.0,155.0
1,2,2_100,1500000US130950001002,2013.0,1.309500e+11,100.0,2,Dougherty County,232.0,0.0,...,0.0,114.0,0.0,0.0,0.0,256.0,194.0,62.0,93.0,101.0
2,3,3_100,1500000US130950001003,2013.0,1.309500e+11,100.0,3,Dougherty County,375.0,67.0,...,0.0,0.0,0.0,0.0,0.0,668.0,504.0,164.0,73.0,431.0
3,4,4_100,1500000US130950001004,2013.0,1.309500e+11,100.0,4,Dougherty County,642.0,41.0,...,0.0,5.0,0.0,0.0,0.0,737.0,635.0,102.0,121.0,514.0
4,5,5_100,1500000US130950001005,2013.0,1.309500e+11,100.0,5,Dougherty County,207.0,0.0,...,0.0,11.0,0.0,0.0,0.0,336.0,263.0,73.0,121.0,142.0


In [10]:
df_tract.head()

,GEO.id,GEO.id2,tract,county,Year,HTotal,FTotal,MTotal,NfTotal,HIncomeLessThan10k,...,EmpRatioEdu25To64HS,UnempRateEdu25To64HS,TotalEdu25To64SomeCollege,LFPRateEdu25To64SomeCollege,EmpRatioEdu25To64SomeCollege,UnempRateEdu25To64SomeCollege,TotalEdu25To64Bachelors,LFPRateEdu25To64Bachelors,EmpRatioEdu25To64Bachelors,UnempRateEdu25To64Bachelors
0,1400000US13007960200,1.300796e+10,960200.0,Baker County,2017.0,517.0,311.0,253.0,206.0,10.6,...,69.0,0.6,214.0,76.6,76.6,0.0,42.0,100.0,100.0,0.0
1,1400000US13007960100,1.300796e+10,960100.0,Baker County,2017.0,804.0,528.0,345.0,276.0,21.9,...,61.9,5.6,331.0,71.9,68.6,4.6,136.0,74.3,74.3,0.0
2,1400000US13321950600,1.332195e+10,950600.0,Worth County,2017.0,854.0,653.0,591.0,201.0,4.1,...,58.6,10.0,450.0,66.7,61.8,7.3,99.0,91.9,91.9,0.0
3,1400000US13321950500,1.332195e+10,950500.0,Worth County,2017.0,1674.0,1135.0,642.0,539.0,18.2,...,62.8,1.0,569.0,86.8,84.9,2.2,175.0,93.7,84.6,9.8
4,1400000US13321950400,1.332195e+10,950400.0,Worth County,2017.0,1573.0,1163.0,949.0,410.0,7.0,...,65.5,10.1,705.0,76.3,70.5,7.6,247.0,77.3,72.5,6.3


In [11]:
df_blockgroup.head()

,BlockGroupID,BlockGroupCombo,GEO.id,Year,GEO.id2,Tract,BlockGroup,County,TotalMale,MaleUnder5,...,HispanicPacificIslander,HispanicOtherRace,Hispanic2Plus,Hispanic2PlusIncOther,Hispanic2PlusExcOther,TotalHomes,TotalOccupiedHomes,TotalVacantHomes,TotalOwnedHomes,TotalRentedHomes
0,1,1_100,1500000US130950001001,2013.0,1.309500e+11,100.0,1,Dougherty County,807.0,58.0,...,0.0,0.0,0.0,0.0,0.0,512.0,512.0,0.0,357.0,155.0
1,2,2_100,1500000US130950001002,2013.0,1.309500e+11,100.0,2,Dougherty County,232.0,0.0,...,0.0,114.0,0.0,0.0,0.0,256.0,194.0,62.0,93.0,101.0
2,3,3_100,1500000US130950001003,2013.0,1.309500e+11,100.0,3,Dougherty County,375.0,67.0,...,0.0,0.0,0.0,0.0,0.0,668.0,504.0,164.0,73.0,431.0
3,4,4_100,1500000US130950001004,2013.0,1.309500e+11,100.0,4,Dougherty County,642.0,41.0,...,0.0,5.0,0.0,0.0,0.0,737.0,635.0,102.0,121.0,514.0
4,5,5_100,1500000US130950001005,2013.0,1.309500e+11,100.0,5,Dougherty County,207.0,0.0,...,0.0,11.0,0.0,0.0,0.0,336.0,263.0,73.0,121.0,142.0


In [77]:
df_tract.head()

,GEO.id,GEO.id2,tract,county,Year,HTotal,FTotal,MTotal,NfTotal,HIncomeLessThan10k,...,EmpRatioEdu25To64HS,UnempRateEdu25To64HS,TotalEdu25To64SomeCollege,LFPRateEdu25To64SomeCollege,EmpRatioEdu25To64SomeCollege,UnempRateEdu25To64SomeCollege,TotalEdu25To64Bachelors,LFPRateEdu25To64Bachelors,EmpRatioEdu25To64Bachelors,UnempRateEdu25To64Bachelors
0,1400000US13007960200,1.300796e+10,960200.0,Baker County,2017.0,517.0,311.0,253.0,206.0,10.6,...,69.0,0.6,214.0,76.6,76.6,0.0,42.0,100.0,100.0,0.0
1,1400000US13007960100,1.300796e+10,960100.0,Baker County,2017.0,804.0,528.0,345.0,276.0,21.9,...,61.9,5.6,331.0,71.9,68.6,4.6,136.0,74.3,74.3,0.0
2,1400000US13321950600,1.332195e+10,950600.0,Worth County,2017.0,854.0,653.0,591.0,201.0,4.1,...,58.6,10.0,450.0,66.7,61.8,7.3,99.0,91.9,91.9,0.0
3,1400000US13321950500,1.332195e+10,950500.0,Worth County,2017.0,1674.0,1135.0,642.0,539.0,18.2,...,62.8,1.0,569.0,86.8,84.9,2.2,175.0,93.7,84.6,9.8
4,1400000US13321950400,1.332195e+10,950400.0,Worth County,2017.0,1573.0,1163.0,949.0,410.0,7.0,...,65.5,10.1,705.0,76.3,70.5,7.6,247.0,77.3,72.5,6.3


In [78]:
df_tract['tract'] = df_tract['tract'].astype(int)
df_blockgroup['Tract'] = df_blockgroup['Tract'].astype('Int64')
df_tract.insert(0, 'TractID', [i for i in range(1, len(df_tract)+1)])
lis = [345, "", np.nan, -1, ""]
lis.extend([np.nan]*270)
not_found_entry = pd.DataFrame([lis], columns=list(df_tract))
df_tract = df_tract.append(not_found_entry, ignore_index=True)

In [79]:
df_tract.tail()

,TractID,GEO.id,GEO.id2,tract,county,Year,HTotal,FTotal,MTotal,NfTotal,...,EmpRatioEdu25To64HS,UnempRateEdu25To64HS,TotalEdu25To64SomeCollege,LFPRateEdu25To64SomeCollege,EmpRatioEdu25To64SomeCollege,UnempRateEdu25To64SomeCollege,TotalEdu25To64Bachelors,LFPRateEdu25To64Bachelors,EmpRatioEdu25To64Bachelors,UnempRateEdu25To64Bachelors
340,341,1400000US13095010401,1.309501e+10,10401,Dougherty County,2010.0,2404.0,1919.0,1648.0,485.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341,342,1400000US13095010302,1.309501e+10,10302,Dougherty County,2010.0,1742.0,1073.0,377.0,669.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,343,1400000US13095001000,1.309500e+10,1000,Dougherty County,2010.0,715.0,419.0,184.0,296.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343,344,1400000US13095000100,1.309500e+10,100,Dougherty County,2010.0,2221.0,1287.0,475.0,934.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,345,,NaN,-1,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
junction_l = []
for i in range(len(df_blockgroup)):
    if i % 10000 == 0:
        print(i)
    block_record = df_blockgroup.iloc[i]
    curr_combo = block_record['Tract']
    allmatches = df_tract['tract'][df_tract['tract'] == curr_combo].index
    if len(allmatches) == 0:
        junction_l.append(([i+1], [345]))
    else:
        junction_l.append((([i+1]*len(allmatches)), allmatches))
junction_one = []
junction_two = []
for i in range(len(junction_l)):
    to_append = junction_l[i]
    junction_one.append(to_append[0])
    junction_two.append(to_append[1])
junction_one = [x for lst in junction_one for x in lst]
junction_two = [x for lst in junction_two for x in lst]
junction_t = pd.DataFrame({'BlockGroupID':junction_one, 'TractID':junction_two})

0


In [83]:
junction.to_csv(path+'TestDB/Ship/AddrJunctBG.csv', index=False)
junction_t.to_csv(path+'TestDB/Ship/BGJunctTract.csv', index=False)

In [86]:
len(junction_t['BlockGroupID'].value_counts())

556

In [87]:
df_tract.tail()

,TractID,GEO.id,GEO.id2,tract,county,Year,HTotal,FTotal,MTotal,NfTotal,...,EmpRatioEdu25To64HS,UnempRateEdu25To64HS,TotalEdu25To64SomeCollege,LFPRateEdu25To64SomeCollege,EmpRatioEdu25To64SomeCollege,UnempRateEdu25To64SomeCollege,TotalEdu25To64Bachelors,LFPRateEdu25To64Bachelors,EmpRatioEdu25To64Bachelors,UnempRateEdu25To64Bachelors
340,341,1400000US13095010401,1.309501e+10,10401,Dougherty County,2010.0,2404.0,1919.0,1648.0,485.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341,342,1400000US13095010302,1.309501e+10,10302,Dougherty County,2010.0,1742.0,1073.0,377.0,669.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,343,1400000US13095001000,1.309500e+10,1000,Dougherty County,2010.0,715.0,419.0,184.0,296.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343,344,1400000US13095000100,1.309500e+10,100,Dougherty County,2010.0,2221.0,1287.0,475.0,934.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,345,,NaN,-1,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
junction_t[junction_t['BlockGroupID'] == 345].shape

(8, 2)

In [91]:
len(junction_t['TractID'].value_counts())

345

In [92]:
len(df_tract)

345